# First Script: Creating the Meta-DataFrame
In this script the author analyzes each column of the synthetic BPI_C 2019 event logs.

In [37]:
#import libraries
import json
import os
import pandas as pd
import numpy
import re

In [38]:
event_log_name = 'BPI_C_2019_1_final_synthetic' #addd the name of the event log - change for each event log
event_log_file_path = f'event_logs/{event_log_name}.csv' #string interpolation, one has to create a folder called 'event_logs' to import the event log.csv
print(event_log_file_path) # review the path

event_logs/BPI_C_2019_1_final_synthetic.csv


In [39]:
#import the event log
event_log_table = pd.read_csv(event_log_file_path) #optionally this can be added; header = 0, encoding = 'ISO-8859-1') if encoding issues arise
event_log_table.head(20) #view the first 20 rows

,eventID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,case Vendor,case Item Type,case Item Category,...,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event concept:name,event Cumulative net worth (EUR),event time:timestamp
0,6.578170e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,20,4507004931_00020,True,NONE,NONE,Vendor creates invoice,68,26-01-1948 23:59:00.000
1,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,10,4507004931_00010,True,NONE,NONE,Vendor creates debit memo,325,26-01-1948 23:59:00.000
2,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,10,4507004931_00010,True,NONE,NONE,Vendor creates invoice,325,26-01-1948 23:59:00.000
3,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,50,4507004931_00050,True,NONE,NONE,Vendor creates invoice,102,26-01-1948 23:59:00.000
4,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,50,4507004931_00050,True,NONE,NONE,Vendor creates debit memo,102,26-01-1948 23:59:00.000
5,6.579030e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,40,4507004931_00040,True,NONE,NONE,Vendor creates debit memo,72,26-01-1948 23:59:00.000
6,6.579030e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,40,4507004931_00040,True,NONE,NONE,Vendor creates invoice,72,26-01-1948 23:59:00.000
7,6.578600e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,30,4507004931_00030,True,NONE,NONE,Vendor creates debit memo,1856,26-01-1948 23:59:00.000
8,6.578600e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,30,4507004931_00030,True,NONE,NONE,Vendor creates invoice,1856,26-01-1948 23:59:00.000
9,6.578170e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,vendor_0645,False,20,4507004931_00020,True,NONE,NONE,Vendor creates debit memo,68,26-01-1948 23:59:00.000


In [40]:
# Modification 1: Add a caseID column, that is defined as a merge of two other columns based on the requirements from the TFPM webpage
'''
  The case ID is a combination of the purchase document and the purchase item.
  https://icpmconference.org/2019/icpm-2019/contests-challenges/bpi-challenge-2019/
'''
#create the caseID and add it to the event log table
event_log_table['caseID'] = event_log_table['case Purchasing Document'].astype('string') + ' ' + event_log_table['case Item'].astype('string')

In [41]:
#review changes on the last column
event_log_table

,eventID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,case Vendor,case Item Type,case Item Category,...,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event concept:name,event Cumulative net worth (EUR),event time:timestamp,caseID
0,6.578170e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,False,20,4507004931_00020,True,NONE,NONE,Vendor creates invoice,68,26-01-1948 23:59:00.000,4507004931 20
1,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,False,10,4507004931_00010,True,NONE,NONE,Vendor creates debit memo,325,26-01-1948 23:59:00.000,4507004931 10
2,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,False,10,4507004931_00010,True,NONE,NONE,Vendor creates invoice,325,26-01-1948 23:59:00.000,4507004931 10
3,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,False,50,4507004931_00050,True,NONE,NONE,Vendor creates invoice,102,26-01-1948 23:59:00.000,4507004931 50
4,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,vendorID_0670,Standard,"3-way match, invoice before GR",...,False,50,4507004931_00050,True,NONE,NONE,Vendor creates debit memo,102,26-01-1948 23:59:00.000,4507004931 50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349519,2.974690e+14,Sales,companyID_0000,Standard PO,Products for Resale,4507020819,Purchase order,vendorID_0115,Standard,"3-way match, invoice before GR",...,False,110,4507020819_00110,True,user_019,user_019,Record Invoice Receipt,1,10-04-2018 10:18:00.000,4507020819 110
349520,3.137560e+14,Specialty Resins,companyID_0000,Standard PO,Alkyd Resins,4507021814,Purchase order,vendorID_0259,Standard,"3-way match, invoice before GR",...,False,10,4507021814_00010,True,user_121,user_121,Create Purchase Order Item,56869,10-04-2018 10:18:00.000,4507021814 10
349521,2.933720e+14,Packaging,companyID_0000,Standard PO,Labels,4507020509,Purchase order,vendorID_0103,Standard,"3-way match, invoice before GR",...,False,170,4507020509_00170,True,user_020,user_020,Record Invoice Receipt,123,10-04-2018 10:18:00.000,4507020509 170
349522,2.974260e+14,Sales,companyID_0000,Standard PO,Products for Resale,4507020819,Purchase order,vendorID_0115,Standard,"3-way match, invoice before GR",...,False,10,4507020819_00010,True,user_019,user_019,Record Invoice Receipt,187,10-04-2018 10:18:00.000,4507020819 10


In [42]:
event_log_table.columns

Index(['eventID ', 'case Spend area text', 'case Company',
       'case Document Type', 'case Sub spend area text',
       'case Purchasing Document', 'case Purch. Doc. Category name',
       'case Vendor', 'case Item Type', 'case Item Category',
       'case Spend classification text', 'case Source', 'case Name',
       'case GR-Based Inv. Verif.', 'case Item', 'case concept:name',
       'case Goods Receipt', 'event User', 'event org:resource',
       'event concept:name', 'event Cumulative net worth (EUR)',
       'event time:timestamp', 'caseID'],
      dtype='object')

In [43]:
#Review all og the event log columns, to decide which ones are mandatory and which ones are additional

log_columns = event_log_table.columns
log_columns # this variable is important for the Meta-DataFrame generation

Index(['eventID ', 'case Spend area text', 'case Company',
       'case Document Type', 'case Sub spend area text',
       'case Purchasing Document', 'case Purch. Doc. Category name',
       'case Vendor', 'case Item Type', 'case Item Category',
       'case Spend classification text', 'case Source', 'case Name',
       'case GR-Based Inv. Verif.', 'case Item', 'case concept:name',
       'case Goods Receipt', 'event User', 'event org:resource',
       'event concept:name', 'event Cumulative net worth (EUR)',
       'event time:timestamp', 'caseID'],
      dtype='object')

In [44]:
# This has to be tailored to each event log
#caseID is created from 'case Purchasing Document' and 'case Item', define the mandatory columns based on domain knowledge analysis

mandatory_columns = ['caseID', 'event time:timestamp', 'event concept:name', 'eventID ']

#Create an object/dictionary and corresponding key:value pairs to add the semantic titles which the csv titles represent in a variable  which are relevant for the Meta-DataFrame
titles = {
  'caseID': 'Case ID',
  'event concept:name': 'Activity',
  'eventID ': 'Event ID',
  'event time:timestamp': 'Timestamp',
}

print(mandatory_columns)

['caseID', 'event time:timestamp', 'event concept:name', 'eventID ']


In [45]:
#Review titles object which are relevant for the Meta-DataFrame
titles

{'caseID': 'Case ID',
 'event concept:name': 'Activity',
 'eventID ': 'Event ID',
 'event time:timestamp': 'Timestamp'}

In [46]:
#Review the datatypes for datatype harmonization
event_log_table.dtypes

eventID                             float64
case Spend area text                 object
case Company                         object
case Document Type                   object
case Sub spend area text             object
case Purchasing Document              int64
case Purch. Doc. Category name       object
case Vendor                          object
case Item Type                       object
case Item Category                   object
case Spend classification text       object
case Source                          object
case Name                            object
case GR-Based Inv. Verif.              bool
case Item                             int64
case concept:name                    object
case Goods Receipt                     bool
event User                           object
event org:resource                   object
event concept:name                   object
event Cumulative net worth (EUR)      int64
event time:timestamp                 object
caseID                          

In [47]:
#This step is optional to minimize the work needed in harmonization

#Translate object data types of timestamp data into a datetime64[ns] data type for the timestamp. This is a unique datatype from pandas.
timestamp_objects = event_log_table['event time:timestamp'] #declare timestamp to a variable
event_log_table['event time:timestamp']=pd.to_datetime(timestamp_objects)

#Review changes made
#event_log_table['event time:timestamp'].dtypes
event_log_table.dtypes

eventID                                    float64
case Spend area text                        object
case Company                                object
case Document Type                          object
case Sub spend area text                    object
case Purchasing Document                     int64
case Purch. Doc. Category name              object
case Vendor                                 object
case Item Type                              object
case Item Category                          object
case Spend classification text              object
case Source                                 object
case Name                                   object
case GR-Based Inv. Verif.                     bool
case Item                                    int64
case concept:name                           object
case Goods Receipt                            bool
event User                                  object
event org:resource                          object
event concept:name             

In [48]:
#This step is optional to minimize the work needed in harmonization 

#Translate object data types of object data types (which contains different possible formats) to a clear string data type format
#for loops where used as they enable somewhat automated transformations 
# = for assigning values
# == operator is used to test equality
for log_column in log_columns:
    if event_log_table[log_column].dtype == 'object': #check if the column has an object data type
        event_log_table[log_column] = event_log_table[log_column].astype('string') #change it to a string

#Review changes
#event_log_table['event time:timestamp'].dtypes
event_log_table.dtypes


eventID                                    float64
case Spend area text                        string
case Company                                string
case Document Type                          string
case Sub spend area text                    string
case Purchasing Document                     int64
case Purch. Doc. Category name              string
case Vendor                                 string
case Item Type                              string
case Item Category                          string
case Spend classification text              string
case Source                                 string
case Name                                   string
case GR-Based Inv. Verif.                     bool
case Item                                    int64
case concept:name                           string
case Goods Receipt                            bool
event User                                  string
event org:resource                          string
event concept:name             

In [49]:
# Reorder the columns of the event log table.
# Checking the log columns list contains each item on the mandatory columns list with the assert statement raises an 'AssertionError' if any of the elements are missing. 
# With the delete method and the == operator, the items are removed in the mandatory columns list from the log columns list.
# Then  using the + operator to combine the mandatory columns list and the log columns list (which has been turned into a list using the tolist method). By doing so, a fresh list of columns in # the desired order is created. Using the [] operator to select only the columns in the newly created list, subsetting the event log table DataFrame. By doing this, the columns in the new #DataFrame are created in the desired order.

for mandatory_column in mandatory_columns:
  assert mandatory_column in log_columns 
  log_columns = log_columns.delete(log_columns == mandatory_column)
log_columns = mandatory_columns + log_columns.tolist()

event_log_table = event_log_table[log_columns]
event_log_table #the for loop above resultes in a move of the mandatory columns to the left side (changes order)

,caseID,event time:timestamp,event concept:name,eventID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event Cumulative net worth (EUR)
0,4507004931 20,1948-01-26 23:59:00,Vendor creates invoice,6.578170e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,20,4507004931_00020,True,NONE,NONE,68
1,4507004931 10,1948-01-26 23:59:00,Vendor creates debit memo,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,10,4507004931_00010,True,NONE,NONE,325
2,4507004931 10,1948-01-26 23:59:00,Vendor creates invoice,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,10,4507004931_00010,True,NONE,NONE,325
3,4507004931 50,1948-01-26 23:59:00,Vendor creates invoice,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,50,4507004931_00050,True,NONE,NONE,102
4,4507004931 50,1948-01-26 23:59:00,Vendor creates debit memo,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,50,4507004931_00050,True,NONE,NONE,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349519,4507020819 110,2018-10-04 10:18:00,Record Invoice Receipt,2.974690e+14,Sales,companyID_0000,Standard PO,Products for Resale,4507020819,Purchase order,...,NPR,sourceSystemID_0000,vendor_0115,False,110,4507020819_00110,True,user_019,user_019,1
349520,4507021814 10,2018-10-04 10:18:00,Create Purchase Order Item,3.137560e+14,Specialty Resins,companyID_0000,Standard PO,Alkyd Resins,4507021814,Purchase order,...,PR,sourceSystemID_0000,vendor_0252,False,10,4507021814_00010,True,user_121,user_121,56869
349521,4507020509 170,2018-10-04 10:18:00,Record Invoice Receipt,2.933720e+14,Packaging,companyID_0000,Standard PO,Labels,4507020509,Purchase order,...,PR,sourceSystemID_0000,vendor_0103,False,170,4507020509_00170,True,user_020,user_020,123
349522,4507020819 10,2018-10-04 10:18:00,Record Invoice Receipt,2.974260e+14,Sales,companyID_0000,Standard PO,Products for Resale,4507020819,Purchase order,...,NPR,sourceSystemID_0000,vendor_0115,False,10,4507020819_00010,True,user_019,user_019,187


Here we are going to reset the index and reorder the event log with a new index

Setting the inplace parameter to True and using the reset index method to reset the DataFrame's index. 
This adds a new column to the DataFrame containing the old index values and resets the index to a range of sequential integers beginning at 0. 
Using the rename method and the inplace parameter set to True, rename the column containing the previous index values. 
A dictionary that maps the old column name "index" to the new column name "EventIndex" makes up the columns parameter.
Using the set index method and the inplace parameter set to True, the DataFrame's index is changed to the 'EventIndex' column. 
This will remove the "EventIndex" column from the data and make it the new index of the DataFrame.

In [50]:
# Number every single event in the event log.
# Every single event is represented by a line in the event log file.
event_log_table.reset_index(inplace=True)
event_log_table.rename(columns={'index': 'EventIndex'}, inplace=True)
event_log_table.set_index('EventIndex', inplace=True)

#review changes
event_log_table

/var/folders/vz/snd73j8d2mj817493bt9ykfc0000gn/T/ipykernel_98011/2322881527.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_log_table.rename(columns={'index': 'EventIndex'}, inplace=True)


,caseID,event time:timestamp,event concept:name,eventID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event Cumulative net worth (EUR)
EventIndex,,,,,,,,,,,,,,,,,,,,,
0,4507004931 20,1948-01-26 23:59:00,Vendor creates invoice,6.578170e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,20,4507004931_00020,True,NONE,NONE,68
1,4507004931 10,1948-01-26 23:59:00,Vendor creates debit memo,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,10,4507004931_00010,True,NONE,NONE,325
2,4507004931 10,1948-01-26 23:59:00,Vendor creates invoice,6.577740e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,10,4507004931_00010,True,NONE,NONE,325
3,4507004931 50,1948-01-26 23:59:00,Vendor creates invoice,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,50,4507004931_00050,True,NONE,NONE,102
4,4507004931 50,1948-01-26 23:59:00,Vendor creates debit memo,6.579460e+13,Sales,companyID_0000,Standard PO,Products for Resale,4507004931,Purchase order,...,NPR,sourceSystemID_0000,vendor_0645,False,50,4507004931_00050,True,NONE,NONE,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349519,4507020819 110,2018-10-04 10:18:00,Record Invoice Receipt,2.974690e+14,Sales,companyID_0000,Standard PO,Products for Resale,4507020819,Purchase order,...,NPR,sourceSystemID_0000,vendor_0115,False,110,4507020819_00110,True,user_019,user_019,1
349520,4507021814 10,2018-10-04 10:18:00,Create Purchase Order Item,3.137560e+14,Specialty Resins,companyID_0000,Standard PO,Alkyd Resins,4507021814,Purchase order,...,PR,sourceSystemID_0000,vendor_0252,False,10,4507021814_00010,True,user_121,user_121,56869
349521,4507020509 170,2018-10-04 10:18:00,Record Invoice Receipt,2.933720e+14,Packaging,companyID_0000,Standard PO,Labels,4507020509,Purchase order,...,PR,sourceSystemID_0000,vendor_0103,False,170,4507020509_00170,True,user_020,user_020,123


In [51]:
#defining a new dictionary  overview_columns with log columns as the elements
content_overview_columns = [(log_column, ['BPI_C_2019_1st_Supplier']) for log_column in log_columns]
extra_overview_columns = [('Event log column property', ['BPI_C_2019_1st_Supplier'])]
overview_columns = dict(extra_overview_columns + content_overview_columns)
overview_table = pd.DataFrame(overview_columns)

# To add a vertically indented name to the index in the console output.
overview_table.set_index('Event log column property', inplace=True)
overview_columns

{'Event log column property': ['BPI_C_2019_1st_Supplier'],
 'caseID': ['BPI_C_2019_1st_Supplier'],
 'event time:timestamp': ['BPI_C_2019_1st_Supplier'],
 'event concept:name': ['BPI_C_2019_1st_Supplier'],
 'eventID ': ['BPI_C_2019_1st_Supplier'],
 'case Spend area text': ['BPI_C_2019_1st_Supplier'],
 'case Company': ['BPI_C_2019_1st_Supplier'],
 'case Document Type': ['BPI_C_2019_1st_Supplier'],
 'case Sub spend area text': ['BPI_C_2019_1st_Supplier'],
 'case Purchasing Document': ['BPI_C_2019_1st_Supplier'],
 'case Purch. Doc. Category name': ['BPI_C_2019_1st_Supplier'],
 'case Vendor': ['BPI_C_2019_1st_Supplier'],
 'case Item Type': ['BPI_C_2019_1st_Supplier'],
 'case Item Category': ['BPI_C_2019_1st_Supplier'],
 'case Spend classification text': ['BPI_C_2019_1st_Supplier'],
 'case Source': ['BPI_C_2019_1st_Supplier'],
 'case Name': ['BPI_C_2019_1st_Supplier'],
 'case GR-Based Inv. Verif.': ['BPI_C_2019_1st_Supplier'],
 'case Item': ['BPI_C_2019_1st_Supplier'],
 'case concept:name': 

In [52]:
#view the new overview_table to construct the Meta-DataFrame
overview_table 

,caseID,event time:timestamp,event concept:name,eventID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event Cumulative net worth (EUR)
Event log column property,,,,,,,,,,,,,,,,,,,,,
BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,...,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier


<h2>Multiple variables based on event data proprocessing which are used for the Meta-DataFrame</h2>

The author uses the pandas .loc function which is an indexer in Pandas is used to choose rows and columns from a DataFrame based on labels.

The snippet below is called title property X as it will be added to the overview table (Meta-DataFrame) with the log columns list's elements as keys and the log column itself as a value, if the title associated with the log column is present in the titles dictionary (if it does not exist in the titles dictionary). 

Then, a new row with the label "Title" and the values from the title property dictionary is added to the overview table DataFrame. The assignment will add the new row to the DataFrame, and the.loc indexer will be used to select the row by its label. 

When this code block is run, a new row with the label "Title" and values representing the names of each log column will appear in the overview table DataFrame (or the log column itself if no title is provided).

In [53]:
# Overview for property: Titles - For the Meta-DataFrame
# Depending on whether the title for the log column is present in the titles dictionary, the values of the log columns list are either the log column itself or its associated title (if it does not exist in the titles dictionary).
title_property = {}

for log_column in log_columns:
    if log_column in titles:
        title = titles[log_column]
    else:
        title = log_column
    title_property[log_column] = title


Then, a new row with the label "Title" and the values from the title property dictionary is added to the overview table DataFrame. The assignment will add the new row to the DataFrame, and the.loc indexer will be used to select the row by its label. 
When this code block is run, a new row with the label "Title" and values representing the names of each log column will appear in the overview table DataFrame (or the log column itself if no title is provided).

In [54]:
overview_table.loc['Title'] = title_property
overview_table.loc['Title']

caseID                                                       Case ID
event time:timestamp                                       Timestamp
event concept:name                                          Activity
eventID                                                     Event ID
case Spend area text                            case Spend area text
case Company                                            case Company
case Document Type                                case Document Type
case Sub spend area text                    case Sub spend area text
case Purchasing Document                    case Purchasing Document
case Purch. Doc. Category name        case Purch. Doc. Category name
case Vendor                                              case Vendor
case Item Type                                        case Item Type
case Item Category                                case Item Category
case Spend classification text        case Spend classification text
case Source                       

<h2>Dictionary {for mandatory and additional attributes}</h2>

This code block generates a new dictionary called importance property whose keys are the items in the list of log columns and whose values are either "Mandatory attribute" or "Additional attribute" depending on whether the log column is included in the mandatory columns list or not. 
Then, a new row with the label "Importance" and values taken from the importance property dictionary is added to the overview table DataFrame. The assignment will add the new row to the DataFrame, and the.loc indexer will be used to select the row by its label. 
When this code block is run, a new row with the label "Importance" and the values "Mandatory attribute" or "Additional attribute" for each log column, depending on whether that column is in the mandatory columns list or not, will appear in the overview table DataFrame.

The concept is used for other data attributes as well

In [55]:
# Overview property: Label for columns either 'Mandatory attribute' or 'additional_attributes' - For the Meta-DataFrame

importance_property = {}
for log_column in log_columns:
    if log_column in mandatory_columns:
        importance_property[log_column] = 'Mandatory attribute'
    else:
        importance_property[log_column] = 'Additional attribute'

overview_table.loc['Importance'] = importance_property
overview_table.loc['Importance']


caseID                               Mandatory attribute
event time:timestamp                 Mandatory attribute
event concept:name                   Mandatory attribute
eventID                              Mandatory attribute
case Spend area text                Additional attribute
case Company                        Additional attribute
case Document Type                  Additional attribute
case Sub spend area text            Additional attribute
case Purchasing Document            Additional attribute
case Purch. Doc. Category name      Additional attribute
case Vendor                         Additional attribute
case Item Type                      Additional attribute
case Item Category                  Additional attribute
case Spend classification text      Additional attribute
case Source                         Additional attribute
case Name                           Additional attribute
case GR-Based Inv. Verif.           Additional attribute
case Item                      

The goal of this code snippet is to add a new row to the Meta-Dataframe termed overview table, where the keys are the columns in a list called log columns and the values are strings indicating whether the column is at the "Event level" or the "Case level." The values are determined by examining the column name for the string "event" or "case," respectively.

In [56]:
# Overview property: The level of the event data attribute in the overview table
level_property = {}
for log_column in log_columns:
  if 'event' in log_column:
    level_property[log_column] = 'Event level'
  if 'case' in log_column:
    level_property[log_column] = 'Case level'

overview_table.loc['Level'] = level_property
overview_table.loc['Level']

caseID                               Case level
event time:timestamp                Event level
event concept:name                  Event level
eventID                             Event level
case Spend area text                 Case level
case Company                         Case level
case Document Type                   Case level
case Sub spend area text             Case level
case Purchasing Document             Case level
case Purch. Doc. Category name       Case level
case Vendor                          Case level
case Item Type                       Case level
case Item Category                   Case level
case Spend classification text       Case level
case Source                          Case level
case Name                            Case level
case GR-Based Inv. Verif.            Case level
case Item                            Case level
case concept:name                    Case level
case Goods Receipt                   Case level
event User                          Even

In [57]:
# Overview property: Data type
data_type_property = {}
datatypes = event_log_table.dtypes
for log_column in log_columns:
  data_type_property[log_column] = datatypes[log_column]

overview_table.loc['Data type'] = data_type_property

This snippet adds a new row to the Meta-Dataframe in Pandas, with the keys being the columns in a list called log columns and the values being the data types of the corresponding columns in an event log table DataFrame.

In [58]:
# Overview property: Data type
data_type_property = {}
datatypes = event_log_table.dtypes
for log_column in log_columns:
    data_type_property[log_column] = datatypes[log_column]

overview_table.loc['Data type'] = data_type_property
overview_table.loc['Data type']

caseID                                      string
event time:timestamp                datetime64[ns]
event concept:name                          string
eventID                                    float64
case Spend area text                        string
case Company                                string
case Document Type                          string
case Sub spend area text                    string
case Purchasing Document                     int64
case Purch. Doc. Category name              string
case Vendor                                 string
case Item Type                              string
case Item Category                          string
case Spend classification text              string
case Source                                 string
case Name                                   string
case GR-Based Inv. Verif.                     bool
case Item                                    int64
case concept:name                           string
case Goods Receipt             

In [59]:
# Overview property: Count the number of entries for the Meta-Dataframe
number_of_entries_property = {}
for log_column in log_columns:
    number_of_entries_property[log_column] = len(event_log_table)

overview_table.loc['Number of entries'] = number_of_entries_property
overview_table.loc['Number of entries']

caseID                              349524
event time:timestamp                349524
event concept:name                  349524
eventID                             349524
case Spend area text                349524
case Company                        349524
case Document Type                  349524
case Sub spend area text            349524
case Purchasing Document            349524
case Purch. Doc. Category name      349524
case Vendor                         349524
case Item Type                      349524
case Item Category                  349524
case Spend classification text      349524
case Source                         349524
case Name                           349524
case GR-Based Inv. Verif.           349524
case Item                           349524
case concept:name                   349524
case Goods Receipt                  349524
event User                          349524
event org:resource                  349524
event Cumulative net worth (EUR)    349524
Name: Numbe

In [60]:
# Overview property: Number of unique entries
number_of_unique_entries_property = {}
for log_column in log_columns:
  count = event_log_table[log_column].value_counts().count()
  number_of_unique_entries_property[log_column] = count

overview_table.loc['Number of unique entries'] = number_of_unique_entries_property
overview_table.loc['Number of unique entries']

caseID                              73329
event time:timestamp                41898
event concept:name                     39
eventID                             73538
case Spend area text                   20
case Company                            3
case Document Type                      3
case Sub spend area text              129
case Purchasing Document            21961
case Purch. Doc. Category name          1
case Vendor                          1544
case Item Type                          6
case Item Category                      4
case Spend classification text          3
case Source                             1
case Name                            1480
case GR-Based Inv. Verif.               2
case Item                             321
case concept:name                   73329
case Goods Receipt                      2
event User                            348
event org:resource                    348
event Cumulative net worth (EUR)    12805
Name: Number of unique entries, dt

In [61]:
#Overview property: List of up to 100 unique entries (to grasp an understanding of how the data looks and it is structured and not just metrics)
unique_entries_property = {}
for log_column in log_columns:
    unique_entries = event_log_table[log_column].unique()
    unique_entries = list(unique_entries)
    unique_entries = unique_entries[:100]
    unique_entries_property[log_column] = unique_entries

overview_table.loc['List up to 100 unique entries to view the structure of the event attributes']= unique_entries_property
unique_entries_property

{'caseID': ['4507004931 20',
  '4507004931 10',
  '4507004931 50',
  '4507004931 40',
  '4507004931 30',
  '4507014062 50',
  '4507014062 90',
  '4507014062 80',
  '4507014062 10',
  '4507014062 20',
  '4507014062 70',
  '4507014062 30',
  '4507014062 40',
  '4507014062 60',
  '4507000647 10',
  '4507009522 10',
  '2000000097 1',
  '4507013723 10',
  '4507027474 10',
  '4507012794 10',
  '4507018023 10',
  '4507017103 20',
  '4507017103 10',
  '4507017103 30',
  '2000013555 1',
  '4507032025 30',
  '4507031332 10',
  '4507036538 20',
  '4507030967 10',
  '4507035116 10',
  '4507035114 10',
  '4507037128 110',
  '4507037128 320',
  '4507037128 310',
  '4507037128 300',
  '4507037128 290',
  '4507037128 280',
  '4507037128 270',
  '4507037128 260',
  '4507037128 250',
  '4507037128 240',
  '4507037128 230',
  '4507037128 220',
  '4507037128 210',
  '4507037128 200',
  '4507037128 190',
  '4507037128 180',
  '4507037128 170',
  '4507037128 160',
  '4507037128 150',
  '4507037128 130',
  '

In [62]:
# Overview property: Length of the list of up to 100 unique entries
num_displayed_unique_entries_property = {}
for log_column in log_columns:
  unique_entries = event_log_table[log_column].unique()
  unique_entries = list(unique_entries)
  unique_entries = unique_entries[:100]
  num_displayed_unique_entries_property[log_column] = len(unique_entries)

overview_table.loc['Length of the list of up to 100 unique entries'] = num_displayed_unique_entries_property
num_displayed_unique_entries_property

{'caseID': 100,
 'event time:timestamp': 100,
 'event concept:name': 39,
 'eventID ': 100,
 'case Spend area text': 21,
 'case Company': 3,
 'case Document Type': 3,
 'case Sub spend area text': 100,
 'case Purchasing Document': 100,
 'case Purch. Doc. Category name': 1,
 'case Vendor': 100,
 'case Item Type': 6,
 'case Item Category': 4,
 'case Spend classification text': 4,
 'case Source': 1,
 'case Name': 100,
 'case GR-Based Inv. Verif.': 2,
 'case Item': 100,
 'case concept:name': 100,
 'case Goods Receipt': 2,
 'event User': 100,
 'event org:resource': 100,
 'event Cumulative net worth (EUR)': 100}

In [63]:
# Inspect the overview table
overview_table

,caseID,event time:timestamp,event concept:name,eventID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event Cumulative net worth (EUR)
Event log column property,,,,,,,,,,,,,,,,,,,,,
BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,...,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier
Title,Case ID,Timestamp,Activity,Event ID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event Cumulative net worth (EUR)
Importance,Mandatory attribute,Mandatory attribute,Mandatory attribute,Mandatory attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,...,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute
Level,Case level,Event level,Event level,Event level,Case level,Case level,Case level,Case level,Case level,Case level,...,Case level,Case level,Case level,Case level,Case level,Case level,Case level,Event level,Event level,Event level
Data type,string,datetime64[ns],string,float64,string,string,string,string,int64,string,...,string,string,string,bool,int64,string,bool,string,string,int64
Number of entries,349524,349524,349524,349524,349524,349524,349524,349524,349524,349524,...,349524,349524,349524,349524,349524,349524,349524,349524,349524,349524
Number of unique entries,73329,41898,39,73538,20,3,3,129,21961,1,...,3,1,1480,2,321,73329,2,348,348,12805
List up to 100 unique entries to view the structure of the event attributes,"[4507004931 20, 4507004931 10, 4507004931 50, ...","[1948-01-26T23:59:00.000000000, 1993-05-18T23:...","[Vendor creates invoice, Vendor creates debit ...","[65781700000000.0, 65777400000000.0, 657946000...","[Sales, Trading & End Products, Logistics, Add...","[companyID_0000, companyID_0003, companyID_0001]","[Standard PO, Framework order, EC Purchase order]","[Products for Resale, Trading products (old st...","[4507004931, 4507014062, 4507000647, 450700952...",[Purchase order],...,"[NPR, PR, <NA>, OTHER]",[sourceSystemID_0000],"[vendor_0645, vendor_0415, vendor_0298, vendor...","[False, True]","[20, 10, 50, 40, 30, 90, 80, 70, 60, 1, 110, 3...","[4507004931_00020, 4507004931_00010, 450700493...","[True, False]","[NONE, user_329, user_236, user_124, batch_03,...","[NONE, user_329, user_236, user_124, batch_03,...","[68, 325, 102, 72, 1856, 274, 9978, 2416, 2032..."
Length of the list of up to 100 unique entries,100,100,39,100,21,3,3,100,100,1,...,4,1,100,2,100,100,2,100,100,100


In [64]:
# Overview property: Number of duplicate entries
number_of_duplicate_entries_property = {}
for log_column in log_columns:
  number_of_duplicate_entries_property[log_column] = number_of_entries_property[log_column] - number_of_unique_entries_property[log_column]

overview_table.loc['Number of duplicate entries'] = number_of_duplicate_entries_property
overview_table.loc['Number of duplicate entries']

caseID                              276195
event time:timestamp                307626
event concept:name                  349485
eventID                             275986
case Spend area text                349504
case Company                        349521
case Document Type                  349521
case Sub spend area text            349395
case Purchasing Document            327563
case Purch. Doc. Category name      349523
case Vendor                         347980
case Item Type                      349518
case Item Category                  349520
case Spend classification text      349521
case Source                         349523
case Name                           348044
case GR-Based Inv. Verif.           349522
case Item                           349203
case concept:name                   276195
case Goods Receipt                  349522
event User                          349176
event org:resource                  349176
event Cumulative net worth (EUR)    336719
Name: Numbe

In [65]:
# Overview property: Number of undefined entries
number_of_undefined_entries_property = {}
nan_info = event_log_table.isna().sum()
for log_column in log_columns:
    number_of_undefined_entries_property[log_column] = nan_info[log_column]

overview_table.loc['Number of undefined entries'] = number_of_undefined_entries_property
number_of_undefined_entries_property

{'caseID': 0,
 'event time:timestamp': 0,
 'event concept:name': 0,
 'eventID ': 0,
 'case Spend area text': 2323,
 'case Company': 0,
 'case Document Type': 0,
 'case Sub spend area text': 2323,
 'case Purchasing Document': 0,
 'case Purch. Doc. Category name': 0,
 'case Vendor': 0,
 'case Item Type': 0,
 'case Item Category': 0,
 'case Spend classification text': 2323,
 'case Source': 0,
 'case Name': 0,
 'case GR-Based Inv. Verif.': 0,
 'case Item': 0,
 'case concept:name': 0,
 'case Goods Receipt': 0,
 'event User': 0,
 'event org:resource': 0,
 'event Cumulative net worth (EUR)': 0}

In [66]:
# Overview propperty: Number of undefined entries in percentages
percentage_of_undefined_entries_property = {}
nan_info = event_log_table.isna().sum()
for log_column in log_columns:
    percentage = nan_info[log_column]/len(event_log_table)*100 #len calculates the length of a variable
    percentage = f'{percentage:.2f}'
    percentage_of_undefined_entries_property[log_column] = percentage

overview_table.loc['Percentage of undefined entries'] = percentage_of_undefined_entries_property
percentage_of_undefined_entries_property

{'caseID': '0.00',
 'event time:timestamp': '0.00',
 'event concept:name': '0.00',
 'eventID ': '0.00',
 'case Spend area text': '0.66',
 'case Company': '0.00',
 'case Document Type': '0.00',
 'case Sub spend area text': '0.66',
 'case Purchasing Document': '0.00',
 'case Purch. Doc. Category name': '0.00',
 'case Vendor': '0.00',
 'case Item Type': '0.00',
 'case Item Category': '0.00',
 'case Spend classification text': '0.66',
 'case Source': '0.00',
 'case Name': '0.00',
 'case GR-Based Inv. Verif.': '0.00',
 'case Item': '0.00',
 'case concept:name': '0.00',
 'case Goods Receipt': '0.00',
 'event User': '0.00',
 'event org:resource': '0.00',
 'event Cumulative net worth (EUR)': '0.00'}

In [67]:
overview_table

,caseID,event time:timestamp,event concept:name,eventID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event Cumulative net worth (EUR)
Event log column property,,,,,,,,,,,,,,,,,,,,,
BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,...,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier,BPI_C_2019_1st_Supplier
Title,Case ID,Timestamp,Activity,Event ID,case Spend area text,case Company,case Document Type,case Sub spend area text,case Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,case GR-Based Inv. Verif.,case Item,case concept:name,case Goods Receipt,event User,event org:resource,event Cumulative net worth (EUR)
Importance,Mandatory attribute,Mandatory attribute,Mandatory attribute,Mandatory attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,...,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute
Level,Case level,Event level,Event level,Event level,Case level,Case level,Case level,Case level,Case level,Case level,...,Case level,Case level,Case level,Case level,Case level,Case level,Case level,Event level,Event level,Event level
Data type,string,datetime64[ns],string,float64,string,string,string,string,int64,string,...,string,string,string,bool,int64,string,bool,string,string,int64
Number of entries,349524,349524,349524,349524,349524,349524,349524,349524,349524,349524,...,349524,349524,349524,349524,349524,349524,349524,349524,349524,349524
Number of unique entries,73329,41898,39,73538,20,3,3,129,21961,1,...,3,1,1480,2,321,73329,2,348,348,12805
List up to 100 unique entries to view the structure of the event attributes,"[4507004931 20, 4507004931 10, 4507004931 50, ...","[1948-01-26T23:59:00.000000000, 1993-05-18T23:...","[Vendor creates invoice, Vendor creates debit ...","[65781700000000.0, 65777400000000.0, 657946000...","[Sales, Trading & End Products, Logistics, Add...","[companyID_0000, companyID_0003, companyID_0001]","[Standard PO, Framework order, EC Purchase order]","[Products for Resale, Trading products (old st...","[4507004931, 4507014062, 4507000647, 450700952...",[Purchase order],...,"[NPR, PR, <NA>, OTHER]",[sourceSystemID_0000],"[vendor_0645, vendor_0415, vendor_0298, vendor...","[False, True]","[20, 10, 50, 40, 30, 90, 80, 70, 60, 1, 110, 3...","[4507004931_00020, 4507004931_00010, 450700493...","[True, False]","[NONE, user_329, user_236, user_124, batch_03,...","[NONE, user_329, user_236, user_124, batch_03,...","[68, 325, 102, 72, 1856, 274, 9978, 2416, 2032..."
Length of the list of up to 100 unique entries,100,100,39,100,21,3,3,100,100,1,...,4,1,100,2,100,100,2,100,100,100


In [68]:
# Switching the index of the Meta-DataFrame and table structure
# The current layout of the Meta-Dataframe is transformed, that the columns become the rows, and the rows become the columns.
overview_table = overview_table.T
overview_table.reset_index(inplace = True)
overview_table.rename(columns={'index': 'Property'}, inplace = True)
overview_table

Event log column property,Property,BPI_C_2019_1st_Supplier,Title,Importance,Level,Data type,Number of entries,Number of unique entries,List up to 100 unique entries to view the structure of the event attributes,Length of the list of up to 100 unique entries,Number of duplicate entries,Number of undefined entries,Percentage of undefined entries
0,caseID,BPI_C_2019_1st_Supplier,Case ID,Mandatory attribute,Case level,string,349524,73329,"[4507004931 20, 4507004931 10, 4507004931 50, ...",100,276195,0,0.00
1,event time:timestamp,BPI_C_2019_1st_Supplier,Timestamp,Mandatory attribute,Event level,datetime64[ns],349524,41898,"[1948-01-26T23:59:00.000000000, 1993-05-18T23:...",100,307626,0,0.00
2,event concept:name,BPI_C_2019_1st_Supplier,Activity,Mandatory attribute,Event level,string,349524,39,"[Vendor creates invoice, Vendor creates debit ...",39,349485,0,0.00
3,eventID,BPI_C_2019_1st_Supplier,Event ID,Mandatory attribute,Event level,float64,349524,73538,"[65781700000000.0, 65777400000000.0, 657946000...",100,275986,0,0.00
4,case Spend area text,BPI_C_2019_1st_Supplier,case Spend area text,Additional attribute,Case level,string,349524,20,"[Sales, Trading & End Products, Logistics, Add...",21,349504,2323,0.66
5,case Company,BPI_C_2019_1st_Supplier,case Company,Additional attribute,Case level,string,349524,3,"[companyID_0000, companyID_0003, companyID_0001]",3,349521,0,0.00
6,case Document Type,BPI_C_2019_1st_Supplier,case Document Type,Additional attribute,Case level,string,349524,3,"[Standard PO, Framework order, EC Purchase order]",3,349521,0,0.00
7,case Sub spend area text,BPI_C_2019_1st_Supplier,case Sub spend area text,Additional attribute,Case level,string,349524,129,"[Products for Resale, Trading products (old st...",100,349395,2323,0.66
8,case Purchasing Document,BPI_C_2019_1st_Supplier,case Purchasing Document,Additional attribute,Case level,int64,349524,21961,"[4507004931, 4507014062, 4507000647, 450700952...",100,327563,0,0.00
9,case Purch. Doc. Category name,BPI_C_2019_1st_Supplier,case Purch. Doc. Category name,Additional attribute,Case level,string,349524,1,[Purchase order],1,349523,0,0.00


In [69]:
#Now we will out put this csv in the analysis folder

if not os.path.exists('analysis'):
    os.mkdir('analysis')
overview_table.to_csv(f'analysis/overview_{event_log_name}.csv')

(f'Written: datatype table to analysis/overview_{event_log_name}.csv')

'Written: datatype table to analysis/overview_BPI_C_2019_1_final_synthetic.csv'

In [70]:
#This is an exploratory code snippet checking for the longest case to model the graph in a deperate database to reveal the event activity sequences in a directly follows graph
case_sizes = event_log_table.groupby(['caseID']).size()
#use of argmax() instead of max()
index_of_longest_case = case_sizes.argmax()
caseID_of_longest_case = case_sizes.index[index_of_longest_case] 

# Without the .copy() suffix, we get: A value is trying to be set on a copy of a slice from a DataFrame
case_rows = event_log_table[event_log_table['caseID'] == caseID_of_longest_case].copy()
case_rows.sort_values(by='event time:timestamp', ascending=True, inplace=True)
case_rows.to_csv(f'analysis/longest_case_in_{event_log_name}.csv')

# a csv table is created in the analysis file which is then transformed into Neo4j commands 

# End of Script 1 to generate the Meta-DataFrame
Author: Kyle Smith <br>
Script: For Masterthesis <br>
University: University of Camerino & University of Applied Sciences Northwestern Switzerland